In [24]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multitest import multipletests


def group_anova(geneid, df):
    '''2 way anova'''
    lst_treatment = ['A']*9+['B']*9+['C']*9+['D']*9
    lst_day = ['d3', 'd7', 'd14']*12
    if df.loc[geneid].mean()<=0:
        result = np.array([np.NaN,np.NaN,np.NaN])
    else:
        df_gene = pd.DataFrame(df.loc[geneid])
        df_gene['treatment'] = lst_treatment
        df_gene['day'] = lst_day
        df_melt = pd.melt(df_gene, id_vars=['treatment', 'day'], value_vars=[geneid])
        model = ols('value ~ C(treatment) + C(day) + C(treatment):C(day)', data = df_melt).fit()
        table = sm.stats.anova_lm(model, typ = 2)
        result = np.array(table['PR(>F)'][:3])
    return result

In [21]:
# read log2 normalized expression matrix
dfrlog = pd.read_csv('./../data/rlog_matrix.csv', index_col=0).rename_axis('geneid')

In [22]:
# 2 way anova for all genes
anova_ls = np.empty((len(dfrlog), 3))
for n in range(len(dfrlog)):
    anova_ls[n, :] = group_anova(list(dfrlog.index)[n], dfrlog)
dfpval = pd.DataFrame(anova_ls, index = dfrlog.index, columns = ['treatment', 'day', 'interaction'])

In [33]:
# multiple tests correction for p value
(_, dfpval['adj_treatment'],_, _) = multipletests(dfpval['treatment'], method='bonferroni')

In [45]:
# rank genes by treatment p value and select first 800 genes
grn_genes = dfpval.sort_values(by = 'adj_treatment')[:800].index

In [49]:
# revert log2 normalized value to original value (recommended by PoLoBag)
(2**dfrlog[dfrlog.index.isin(grn_genes)]-0.5).to_csv('./../data/grn_expression.txt', sep='\t')